# Dataset Generation

In [1]:
%cd -q ..

from src.db_utils import *

%cd -q research

## Generation of User-Track-Rating Dataset

### 1. User's Top Tracks

In [4]:
top_tracks = all_top_tracks()
top_tracks

,user_id,track_id,rank
0,1,35151,1
1,1,560787,2
2,1,231167,3
3,1,302447,4
4,1,156880,5
...,...,...,...
1054551,52829,616532,16
1054552,52829,298170,17
1054553,52829,149950,18
1054554,52829,162448,19


In [5]:
PATH = "../data/recsys_data/top_tracks_ratings.csv"

# Convert rank into a rating from 5 to 3.1 / 5
import numpy as np

ratings = np.linspace(5, 3.1, 20, dtype=np.float32)
print(f"Ratings by rank: {ratings}")

top_track_ratings = top_tracks[['user_id', 'track_id']]
top_track_ratings['rating'] = top_tracks['rank'].apply(lambda x: ratings[x-1])

top_track_ratings

Ratings by rank: [5.  4.9 4.8 4.7 4.6 4.5 4.4 4.3 4.2 4.1 4.  3.9 3.8 3.7 3.6 3.5 3.4 3.3
 3.2 3.1]


,user_id,track_id,rating
0,1,35151,5.0
1,1,560787,4.9
2,1,231167,4.8
3,1,302447,4.7
4,1,156880,4.6
...,...,...,...
1054551,52829,616532,3.5
1054552,52829,298170,3.4
1054553,52829,149950,3.3
1054554,52829,162448,3.2


In [6]:
top_track_ratings.to_csv(PATH, index=False, sep='\t')

### 2. User's Recent Tracks

In [6]:
recent_tracks = all_recent_tracks()
recent_tracks

,user_id,track_id,listen_at
0,2,785090,2022-11-12 09:22:07
1,2,273675,2022-11-12 09:27:09
2,2,462697,2022-11-12 09:31:14
3,2,9158,2022-11-12 09:36:13
4,2,180323,2022-11-12 09:39:16
...,...,...,...
1019975,52829,554368,2022-11-15 04:27:34
1019976,52829,745290,2022-11-15 04:32:42
1019977,52829,539622,2022-11-15 04:37:47
1019978,52829,677435,2022-11-15 04:44:16


In [7]:
user_track_counts = recent_tracks.drop(columns='listen_at').groupby('user_id').value_counts()
avg_repeat = user_track_counts.mean()
max_repeat = user_track_counts.max()

print(f"Average repeat listens: {avg_repeat.round(3)}")
print(f"Most repeat listens: {max_repeat}")

Average repeat listens: 1.183
Most repeat listens: 20


In [8]:
import numpy as np
from tqdm import tqdm

PATH = "../data/recsys_data/recent_tracks_ratings.csv"

# Convert heard songs into a rating from 1 to 5 based on listen count
recent_track_ratings = recent_tracks[['user_id', 'track_id']]

# Listen count intervals to assign ratings
limits = list(np.linspace(max_repeat // 2, 1, 5, dtype=int)) + [0]
limits[1], limits[0] = limits[0], max_repeat
ratings = [5, 4, 3, 2, 1]
print(f"Listen count intervals: {limits}")

lim_ratings = list(enumerate(zip(limits[:-1], ratings)))

for user in tqdm(recent_tracks['user_id'].unique(), desc="Assigning ratings per user"):
    user_listen_count = user_track_counts[user]
    user_filter = recent_track_ratings['user_id'].values == user

    for i, (lim, r) in lim_ratings:
        track_ids = user_listen_count[(user_listen_count.values <= lim) & (user_listen_count.values > limits[i+1])].index
        track_intval_filter = user_filter & np.isin(recent_track_ratings['track_id'], track_ids)
        recent_track_ratings.loc[track_intval_filter, 'rating'] = r

recent_track_ratings['timestamp'] = recent_tracks['listen_at']

# Keep only one listen for repeats, the latest one
recent_track_ratings.drop_duplicates(subset=['user_id', 'track_id'], inplace=True, keep='last')

recent_track_ratings

Listen count intervals: [20, 10, 5, 3, 1, 0]


Assigning ratings per user: 100%|██████████| 51129/51129 [09:35<00:00, 88.86it/s] 


,user_id,track_id,rating,timestamp
0,2,785090,1.0,2022-11-12 09:22:07
1,2,273675,1.0,2022-11-12 09:27:09
2,2,462697,1.0,2022-11-12 09:31:14
3,2,9158,1.0,2022-11-12 09:36:13
4,2,180323,1.0,2022-11-12 09:39:16
...,...,...,...,...
1019975,52829,554368,1.0,2022-11-15 04:27:34
1019976,52829,745290,1.0,2022-11-15 04:32:42
1019977,52829,539622,1.0,2022-11-15 04:37:47
1019978,52829,677435,1.0,2022-11-15 04:44:16


In [11]:
recent_track_ratings.to_csv(PATH, index=False, sep='\t')

### 3. User's Loved Tracks

In [9]:
loved_tracks = all_loved_tracks()
loved_tracks

,user_id,track_id,love_at
0,3,762797,2022-06-27 11:29:18
1,3,169241,2022-06-27 11:29:20
2,3,237435,2022-06-27 11:29:29
3,3,708715,2022-06-27 11:29:31
4,3,695146,2022-07-05 09:50:14
...,...,...,...
377621,52829,186736,2022-11-01 14:14:07
377622,52829,30097,2022-11-01 14:14:14
377623,52829,42110,2022-11-01 14:15:02
377624,52829,59138,2022-11-01 14:19:24


In [10]:
PATH = "../data/recsys_data/loved_tracks_ratings.csv"

# Convert heard songs into a rating of 5 / 5
loved_track_ratings = loved_tracks[['user_id', 'track_id']]
loved_track_ratings['rating'] = 5
loved_track_ratings['timestamp'] = loved_tracks['love_at']

loved_track_ratings

,user_id,track_id,rating,timestamp
0,3,762797,5,2022-06-27 11:29:18
1,3,169241,5,2022-06-27 11:29:20
2,3,237435,5,2022-06-27 11:29:29
3,3,708715,5,2022-06-27 11:29:31
4,3,695146,5,2022-07-05 09:50:14
...,...,...,...,...
377621,52829,186736,5,2022-11-01 14:14:07
377622,52829,30097,5,2022-11-01 14:14:14
377623,52829,42110,5,2022-11-01 14:15:02
377624,52829,59138,5,2022-11-01 14:19:24


In [14]:
loved_track_ratings.to_csv(PATH, index=False, sep='\t')

### 4. Merge All Tracks

Merging all three datasets by concatenation. Duplicates are dropped by keeping the highest rating.

In [11]:
PATH = "../data/recsys_data/all_tracks_ratings.csv"
MIN_PLAYCOUNT = 2 # If only 1, useless in colab. filtering recommendation

def merge_tracks(track_dfs, discard_min_playcount=True):
    merged_track_ratings = pd.concat(track_dfs, ignore_index=True)
    merged_track_ratings['rating'] = merged_track_ratings['rating'].astype(np.float32)

    # Discard tracks with less than MIN_PLAYCOUNT total listens
    # NOTE try including all tracks
    if discard_min_playcount:
        listen_count = merged_track_ratings['track_id'].value_counts()
        discard_ids = listen_count[listen_count < MIN_PLAYCOUNT].index
        merged_track_ratings = merged_track_ratings.loc[~merged_track_ratings.track_id.isin(discard_ids)]

    # Sort by desc. rating and drop duplicates while keeping highest rating
    final_ratings = merged_track_ratings.sort_values('rating', ascending=False).drop_duplicates(
        ['user_id', 'track_id'], keep='first').sort_values(['user_id', 'track_id']).reset_index(drop=True)

    return final_ratings


#### Discarding tracks listened only once

In [ ]:
final_ratings = merge_tracks([top_track_ratings, recent_track_ratings, loved_track_ratings])
final_ratings

#### Including all tracks

In [12]:
final_ratings = merge_tracks([top_track_ratings, recent_track_ratings, loved_track_ratings], discard_min_playcount=False)
final_ratings

,user_id,track_id,rating,timestamp
0,1,35151,5.0,NaT
1,1,82497,3.3,NaT
2,1,97967,4.2,NaT
3,1,105492,3.5,NaT
4,1,124021,3.9,NaT
...,...,...,...,...
2200756,52829,711172,4.2,NaT
2200757,52829,745290,1.0,2022-11-15 04:32:42
2200758,52829,776707,1.0,2022-11-14 17:46:56
2200759,52829,781335,1.0,2022-11-14 17:37:05


In [13]:
final_ratings.to_csv(PATH, index=False, sep='\t')

## Generation of Item-VAD Dataset

In [4]:
track_vad = get_item_vad(TRACK, 'Track')
track_vad

,Track,Name,V,A,D,StSc
0,1,twentyseven,NaN,NaN,NaN,NaN
1,2,Happy Xmas (War Is Over) (2003 Mix),0.667880,0.440461,0.568477,0.720910
2,3,Don’t Shut Me Down,0.587845,0.490143,0.504333,0.555556
3,4,Tirpitz,NaN,NaN,NaN,NaN
4,5,You Have Cum in Your Hair and Your Dick is Han...,0.623167,0.380725,0.507896,0.654302
...,...,...,...,...,...,...
815626,815627,D-1,0.677566,0.472298,0.600010,0.875615
815627,815628,Now or Never,NaN,NaN,NaN,NaN
815628,815629,Los Zurdos Mueren Antes,NaN,NaN,NaN,NaN
815629,815630,Oh Mama,0.608476,0.440966,0.554662,0.634783


In [5]:
t_artist_vad = get_track_artist_vads()
t_artist_vad

,Track,V,A,D,StSc
0,2,0.662364,0.453781,0.582089,0.769586
1,4,0.561424,0.483773,0.533182,0.378588
2,8,0.613225,0.473498,0.544577,0.590839
3,9,0.631987,0.476708,0.578362,0.696299
4,12,0.623932,0.446357,0.546906,0.670236
...,...,...,...,...,...
815626,815592,0.655076,0.472305,0.558858,0.744371
815627,815597,0.651401,0.461883,0.575568,0.751494
815628,815606,0.592959,0.474800,0.563727,0.583555
815629,815615,0.636870,0.457241,0.574222,0.684615


In [6]:
t_album_vad = get_track_album_vads()
t_album_vad

,Track,V,A,D,StSc
0,12,0.632310,0.444328,0.556632,0.718256
1,19,NaN,NaN,NaN,NaN
2,25,0.584914,0.483083,0.550804,0.454361
3,30,NaN,NaN,NaN,NaN
4,44,NaN,NaN,NaN,NaN
...,...,...,...,...,...
604199,815586,0.628460,0.463568,0.570361,0.705924
604200,815588,NaN,NaN,NaN,NaN
604201,815589,NaN,NaN,NaN,NaN
604202,815601,NaN,NaN,NaN,NaN


In [7]:
print(f"Tracks with VAD: {track_vad['V'].notna().sum()}")
print(f"Tracks with artist's VAD: {t_artist_vad['V'].notna().sum()}")
print(f"Tracks with album's VAD: {t_album_vad['V'].notna().sum()}")

Tracks with VAD: 361091
Tracks with artist's VAD: 759499
Tracks with album's VAD: 318036


Not worth it to use albums to fill this time, so use only artists

#### Use Artist VADs to fill NaN values for Track

In [8]:
PATH = "../data/recsys_data/all_tracks_vads.csv"

track_isna = track_vad['V'].isna()
nan_tracks = track_vad[track_isna]['Track'].values

merged_tracks = track_vad.copy()
replace_cols = ['V', 'A', 'D', 'StSc']
merged_tracks.loc[merged_tracks['Track'].isin(
    nan_tracks), replace_cols] = t_artist_vad.loc[t_artist_vad['Track'].isin(nan_tracks), replace_cols]

print(f"Final tracks with VAD: {merged_tracks['V'].notna().sum()}")

merged_tracks


Final tracks with VAD: 583710


,Track,Name,V,A,D,StSc
0,1,twentyseven,NaN,NaN,NaN,NaN
1,2,Happy Xmas (War Is Over) (2003 Mix),0.667880,0.440461,0.568477,0.720910
2,3,Don’t Shut Me Down,0.587845,0.490143,0.504333,0.555556
3,4,Tirpitz,0.631987,0.476708,0.578362,0.696299
4,5,You Have Cum in Your Hair and Your Dick is Han...,0.623167,0.380725,0.507896,0.654302
...,...,...,...,...,...,...
815626,815627,D-1,0.677566,0.472298,0.600010,0.875615
815627,815628,Now or Never,0.651401,0.461883,0.575568,0.751494
815628,815629,Los Zurdos Mueren Antes,NaN,NaN,NaN,NaN
815629,815630,Oh Mama,0.608476,0.440966,0.554662,0.634783


#### Normalize with Tag VADs

In [9]:
tags_vad = get_item_vad(TAG, 'Tag')

# Get normalized columns
merged_tracks['V_Norm'] = normalize(merged_tracks['V'], kind='minmax', usecol=tags_vad['V'])
merged_tracks['A_Norm'] = normalize(merged_tracks['A'], kind='minmax', usecol=tags_vad['A'])
merged_tracks['D_Norm'] = normalize(merged_tracks['D'], kind='minmax', usecol=tags_vad['D'])
merged_tracks['StSc_Norm'] = normalize(merged_tracks['StSc'], kind='minmax', usecol=tags_vad['StSc'])

merged_tracks

,Track,Name,V,A,D,StSc,V_Norm,A_Norm,D_Norm,StSc_Norm
0,1,twentyseven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Happy Xmas (War Is Over) (2003 Mix),0.667880,0.440461,0.568477,0.720910,0.571557,0.415936,0.474975,0.860455
2,3,Don’t Shut Me Down,0.587845,0.490143,0.504333,0.555556,0.461959,0.488838,0.363161,0.777778
3,4,Tirpitz,0.631987,0.476708,0.578362,0.696299,0.522406,0.469124,0.492206,0.848150
4,5,You Have Cum in Your Hair and Your Dick is Han...,0.623167,0.380725,0.507896,0.654302,0.510328,0.328283,0.369372,0.827151
...,...,...,...,...,...,...,...,...,...,...
815626,815627,D-1,0.677566,0.472298,0.600010,0.875615,0.584822,0.462653,0.529943,0.937808
815627,815628,Now or Never,0.651401,0.461883,0.575568,0.751494,0.548992,0.447370,0.487336,0.875747
815628,815629,Los Zurdos Mueren Antes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
815629,815630,Oh Mama,0.608476,0.440966,0.554662,0.634783,0.490210,0.416677,0.450893,0.817391


In [ ]:
merged_tracks.to_csv(PATH, index=False, sep='\t')